In [1]:
import sqlite3 


IMPORTANT: You need a Python kernal that's been compiled that has loadable sqlite extensions enabled. If you don't want to compile it yourself, use the package on Homebrew, which comes with the flag set.

https://til.simonwillison.net/sqlite/sqlite-extensions-python-macos 

In [2]:
con = sqlite3.connect("/Users/Bumpant/Development/LLM/Rag/testLembed")
con.enable_load_extension(True)
con.load_extension("../sqlite-lembed/lembed0")
con.load_extension("../sqlite-vec/vec0")

In [3]:
curs = con.cursor()


In [4]:
curs.execute("INSERT INTO temp.lembed_models(name, model) select 'nomic-embed-text-1.5', lembed_model_from_file('/Users/Bumpant/Development/LLM/Rag/nomic-embed-text-v1.5.Q2_K.gguf');")


In [63]:
curs.execute('create table articles(headline text);')

# -- Random NPR headlines from 2024-06-04
curs.execute("insert into articles VALUES('Shohei Ohtani''s ex-interpreter pleads guilty to charges related to gambling and theft'),('The jury has been selected in Hunter Biden''s gun trial'),('Larry Allen, a Super Bowl champion and famed Dallas Cowboy, has died at age 52'),('After saying Charlotte, a lone stingray, was pregnant, aquarium now says she''s sick'),('An Epoch Times executive is facing money laundering charge');");

# -- Build a vector table with embeddings of article headlines, using OpenAI's API
curs.execute('create virtual table vec_articles using vec0(headline_embeddings float[768]);')

curs.execute("insert into vec_articles(rowid, headline_embeddings) select rowid, lembed('nomic-embed-text-1.5', headline) from articles;")



In [64]:
curs.execute("with matches as (select rowid, distance from vec_articles where headline_embeddings match lembed('nomic-embed-text-1.5', 'now says she is sick') order by distance limit 3) select headline, distance from matches left join articles on articles.rowid = matches.rowid;")

print(curs.fetchall())


[("The jury has been selected in Hunter Biden's gun trial", 0.509064257144928), ('Larry Allen, a Super Bowl champion and famed Dallas Cowboy, has died at age 52', 0.5173013806343079), ("After saying Charlotte, a lone stingray, was pregnant, aquarium now says she's sick", 0.5219035744667053)]


In [5]:
con.close()